In [4]:
#imports
from tkinter import *
import os
import re
import time
import datetime

import telnetlib as tn
import getpass 
import sys
import time
import serial
import threading

import pandas as pd
import numpy as np

os.environ.setdefault('MAGICK_HOME', '/usr/local/Cellar/imagemagick@6/6.9.9-34/')
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io



#global defines
nport_ip = "10.136.1.99"
ref_port = "4001"



In [105]:
#generate "main" gui window in tkinter
def onclick(): 
        pass

#updates the main status label with given text.  Could be accomplished by just calling "main_status.config" from within the main function...
def update_txt(next_instruction):
    main_status.config(text=next_instruction)   
    
#def connex(port):
    #print(port)
    
win = Tk()

win.title('SBE CTD Prequalification')
win.geometry('800x600') # Size 200, 200

text = Text(win, height=2)

#labels
main_status = Label(win, text="Waiting for device selection")
calsheet_status_label = Label(win, text="")

port2 = Button(win, text="4002 - 485", command=lambda: instrument_connection("4002"))
port3 = Button(win, text="4003 - 232", command=lambda: instrument_connection("4003"))
port4 = Button(win, text="4004", command=lambda: instrument_connection("4004"))

text.insert(INSERT, "Please connect the RMA device and specify the port used")

text.pack()
port2.pack()
port3.pack()
port4.pack()
main_status.pack()
calsheet_status_label.pack()


#btn = Button(win, text='Hello', command=hello)
#btn.pack(expand=YES, fill=BOTH)

mainloop()


b'SBE37SI-RS485 V 3.2  5679\r\ntemperature:  25-Mar-17\r\n    TA0 = 4.261456e-05\r\n    TA1 = 2.731360e-04\r\n    TA2 = -2.206636e-06\r\n    TA3 = 1.490857e-07\r\nconductivity:  25-Mar-17\r\n    G = -9.994807e-01\r\n    H = 1.566288e-01\r\n    I = -3.317297e-04\r\n    J = 4.905268e-05\r\n    CPCOR = -9.570000e-08\r\n    CTCOR = 3.250000e-06\r\n    WBOTC = 1.185617e-06\r\nreference pressure = 2007.0 decibars\r\n<Executed/>\r\n'


In [91]:
# Instrument Connection
#module to:
# 1. Prompt user to select NPORT port RMA instrument is connected to (maybe goes in main gui window?)
# 2. Create a telnet connection to both instruments
# 3. Send wakeup to both instruments
# 4. Alert user that connection successful

#entry point to module
def instrument_connection(rma_port):
    global nport_ip
    global ref_port
    device_dict = {
        "4002" : "id?\r\n",
        "4003" : "\r\n"
    }
    try:
        ref_telnet = telnetlib.Telnet(nport_ip, ref_port) #open connection
        rma_telnet = telnetlib.Telnet(nport_ip, rma_port)
        update_txt("Connecting, please wait...")
        time.sleep(1)
        ref_telnet.write(device_dict['4003'].encode("ascii"))  #wakeup both instruments
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_ok = rma_telnet.read_very_eager()
        ref_ok = ref_telnet.read_very_eager()
        #print(str(ref_ok))
        #print(str(rma_ok))
        ref_telnet.close() #close reference connection, not done with rma yet
        rma_telnet.close()
    except:
        print("bricked it")
        ref_telnet.close()
        rma_telnet.close()
    
    update_txt("Connection Successful")
    calsheet_status_label.config(text="Collecting RMA calibration data...")    
    compare_cals(rma_port) #pass the telnet connection to compare_cals function    

In [ ]:
# Compare Samples
#module to:
# 1. Collect 10 samples (1/min) from each instrument
# 2. Compare samples at like timestamps(should be +/- 1s)
# 3. Format data and present to user, advise pass/fail, generate comparison column as well

In [104]:
# Compare Cals
#module to:
# 1. Prompt user for calibration sheet location
# 2. Collect cals from instrument(dcal, #iidc)
# 3. Compare Results
# 4. Display side by side results AND suggest an action(pass/fail)
# 5. If "pass" then continue after ~30seconds if "fail" pause and wait for user?\
def compare_cals(rma_port):
    if rma_port == "4002":
        ins_caldata = get_cals_37()
    if rma_port == "4003":
        ins_caldata = get_cals_19()
    sheet_caldata = get_sheet_cals()
    
def get_cals_37():
    global nport_ip
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    sbe_id = str(rma_telnet.read_very_eager())[5:7]
    rma_telnet.write(("#"+sbe_id+"DC\r\n").encode("ascii"))
    time.sleep(1)
    #print(rma_telnet.read_very_eager()) #cal data to be parsed
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return caldata
def get_cals_19():
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("DCal\r\n".encode("ascii"))
    time.sleep(1)
    #print(rma_telnet.read_very_eager()) #cal data to be parsed
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return caldata

def get_sheet_cals():
    

In [ ]:
# Cleanup
# 1. Close telnet connections
# 2. Save any relevant tables/data to files(not sure if any exists)



In [5]:
#noodle tab
def test_func():
    global nport_ip
    test = nport_ip
    return test

print(test_func())

10.136.1.99


In [89]:
try:
    two = 1+1
except:
    three = 1+1
    
print(two)

2
